In [1]:
import json
import pandas as pd
import numpy as np
from emoji import core
import re
import ast
from matplotlib import pyplot as plt

In [156]:
playlists = pd.read_json('../0_data/playlists.json', typ = 'frame', encoding='utf-8')
song_meta = pd.read_json('../0_data/song_meta.json', typ = 'frame', encoding='utf-8') 
genre_gn_all = pd.read_json('../0_data/genre_gn_all.json', typ = 'frame', encoding='utf-8', orient='index')

### 1. song_meta

> 1. 날짜 오류 수정

In [36]:
# 날짜 오류있는 행 확인
song_meta[song_meta['issue_date'] > 20220000] # index : 141185, 448286

indices_to_update = [141185, 448286]
new_value = 20200113 # 알맞는 발매날짜
# 특정 행의 'issue_date' 값을 변경
for idx in indices_to_update:
    song_meta.at[idx, 'issue_date'] = new_value

In [37]:
# 알맞게 변경되었는지 확인
song_meta.iloc[[141185, 448286], :]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
141185,[GN1801],20200113,고요한 밤바다 구경하기 [여수 바다],10403230,[2737142],잔잔한 바람이 불어와 (There is a gentle breeze),[GN1800],[무드홀릭 (Moodholic)],141185
448286,[GN1801],20200113,고요한 밤바다 구경하기 [여수 바다],10403230,[2737142],여수 바다 (Yeosu sea),[GN1800],[무드홀릭 (Moodholic)],448286


> 2. 대분류 장르 GN9000노래 삭제

In [38]:
song_meta[song_meta['song_gn_gnr_basket'].apply(lambda x: 'GN9000' in x)] # 1834개
song_meta = song_meta[~song_meta['song_gn_gnr_basket'].apply(lambda x: 'GN9000' in x)] # 삭제

In [39]:
# 삭제되었는지 확인
song_meta[song_meta['song_gn_gnr_basket'].apply(lambda x: 'GN9000' in x)]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id


In [40]:
# song_meta 저장
song_meta.to_json('../0_data/song_meta_filter.json', orient='records')

### 2. playlists

> 1. 타이틀, 태그 불용어 처리

- 타이틀 특수문자 제거

In [157]:
# 이모지 제거 함수
def replace_emoji(inputString):
    return core.replace_emoji(inputString, replace='')

# 특수문자 제거 함수
def clean_text(inputString):
  text_rmv = re.sub('[-=+,#/\?:^●○.@*\"※~ㆍ!<>$♥❤♡☆★♪♬♩¶♨①๑｡＊②ఇ¥◈ ͡✔ ͡° ͜ʖ▶◀◎ღ✿ﾉ ;\\\_■『│』ᕷⓗ‘|\(\)\[\]`\'…》\”\“\’·]',' ', inputString)
  return text_rmv

In [158]:
# # 이모지 제거 함수 적용
playlists['plylst_title'] = playlists['plylst_title'].apply(replace_emoji)

# 특수문자 제거 함수 적용
playlists['plylst_title'] = playlists['plylst_title'].apply(clean_text)

In [159]:
# 양 옆 공백, %, & 제거

playlists['plylst_title'] = playlists['plylst_title'].str.strip()
playlists['plylst_title'] = playlists['plylst_title'].str.strip('%')
playlists['plylst_title'] = playlists['plylst_title'].str.strip('&')

In [160]:
playlists.iloc[[56134, 59162],:] # 확인

,tags,id,plylst_title,songs,like_cnt,updt_date
56134,[흥폭발],22525,내적댄스 터져베이베 아이돌 노래노래,"[108003, 594076, 297411, 97189, 35713, 445680,...",41,2018-06-15 19:24:26.000
59162,"[가을, 설렘, 사랑]",143039,가을이다 o 곧 겨울이다 T T,"[13142, 5049, 503816, 70166, 578529, 336829, 2...",1,2018-07-31 10:55:04.000


- 태그 특수문자 제거

In [161]:
# 특수문자 '_'가 있는 행 찾기
# '_'가 있는 행 찾기
rows_with_underscore = playlists['tags'][playlists['tags'].apply(lambda x: any('_' in tag for tag in x))]
rows_with_underscore

148                            [비오는날, 비_오는_날, 휴식, 비_오는날, 비]
285                                [상쾌해지는, 호불호없는, 모두가_흥얼흥얼]
364        [추천, 일렉트로니카, 5월_4주차, 주간, 최신, 전지음악, 인기, 주간일렉트로니카]
409                      [들었으면, 인디음악, 한번쯤, 이미지는_직접찍은사진, 인디]
522       [힙합, label_crew, 랩, 국내힙합, 트렌드, 쇼미더머니, 스웨그, aom...
                                ...                        
114625                                   [칠_Chill, 설레는_달콤함]
114647                     [우울, 위로, 위로가_필요할_때, 슬픔, 이별, 힘든날]
114688    [어쿠스틱, 혼자, 발라드, 우울, 분위기, 멜로디, 띵곡들, 조용한, 자기_전, 인디]
114703    [패션위크, 뉴욕, 도시, 갈라쇼음악, 트렌디, 팝, 편집샵, i_know_it, ...
115068               [담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]
Name: tags, Length: 2252, dtype: object

In [162]:
# 특수 문자를 제거하는 함수 정의
def remove_special_characters(tag_list):
    cleaned_tags = []
    for tag in tag_list:
        cleaned_tag = re.sub(r'[^\uAC00-\uD7A30-9a-zA-Z\s]', '', tag).replace(" ", "")
        cleaned_tags.append(cleaned_tag)
    return cleaned_tags

# 함수 적용
playlists['tags'] = playlists['tags'].apply(remove_special_characters)

In [163]:
playlists.iloc[[148, 285, 115068], :] # 확인

,tags,id,plylst_title,songs,like_cnt,updt_date
148,"[비오는날, 비오는날, 휴식, 비오는날, 비]",32533,비오는날 감성 자극 듣기 좋은 어쿠스틱 피아노 발라드,"[133091, 527764, 468136, 260894, 22845, 148876...",14,2020-02-16 00:56:45.000
285,"[상쾌해지는, 호불호없는, 모두가흥얼흥얼]",151357,기분좋아지는 노래들,"[270061, 158766, 573179, 272062, 37027, 245028...",0,2019-11-24 05:49:55.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의이야기, 사랑, 친구]",11343,1 눈물이 앞을 가리는 나의 이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000


In [164]:
# 플레이리스트 타이틀이 1글자인 조건 만들기
playlists_title_1str = playlists[playlists['plylst_title'].str.len() == 1]
playlists_title_1str = list(playlists_title_1str['id'].values) # 24 -> 롹, 잠, 쿨, 힙 등이며 대부분 좋아요 0 또는 1
# playlists[playlists['id'] == 98628]

# 삭제
playlists = playlists[playlists['plylst_title'].str.len() != 1]

In [165]:
# 빈 타이틀 삭제
playlists = playlists[playlists['plylst_title'] != '']

# 숫자로만 구성된 타이틀 삭제
numeric_title_rows = playlists[playlists['plylst_title'].str.isnumeric()]
indices_to_remove = numeric_title_rows.index
playlists = playlists.drop(indices_to_remove)

playlists.info()


# 115071 - 114707 = 364행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114707 entries, 0 to 115070
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          114707 non-null  object
 1   id            114707 non-null  int64 
 2   plylst_title  114707 non-null  object
 3   songs         114707 non-null  object
 4   like_cnt      114707 non-null  int64 
 5   updt_date     114707 non-null  object
dtypes: int64(2), object(4)
memory usage: 6.1+ MB


> 2. 타이틀 중복 제거(기준 : 노래 개수 or 좋아요 개수)

In [166]:
# 1. 플레이리스트 아이디(id)와 플레이리스트 명(plylst_title) 추출
plylst_title = playlists[['id', 'plylst_title']]

# 2. 플레이리스트 이름 별 플레이리스트 ID count 테이블 생성 : plylst_id_cnt
plylst_id_cnt = plylst_title.groupby('plylst_title').id.nunique().reset_index(name = 'mapping_plylst_cnt')

# 3. 플레이리스트 명 중복 구분 : 서로 다른 플레이리스트 아이디로 겹치는 플레이리스트 이름이 없으면 '중복 없음', 그 외 '중복 있음'
plylst_id_cnt = plylst_id_cnt.assign(
    mapping_plylst_cnt_category = pd.cut(plylst_id_cnt['mapping_plylst_cnt'], [0, 1, np.inf], labels = ['중복 없음', '중복 있음'])
)
plylst_id_cnt
# [0, 1, np.inf] : 0이상 1미만 값 = 중복없음, 1이상의 값 = 중복있음

# 4. 중복 구분 별 플레이리스트 수 count 테이블 생성 : plylst_id_cnt_division
plylst_id_cnt_division = pd.DataFrame(plylst_id_cnt.groupby('mapping_plylst_cnt_category').count())
plylst_id_cnt_division

,plylst_title,mapping_plylst_cnt
mapping_plylst_cnt_category,,
중복 없음,106525,106525
중복 있음,2722,2722


In [167]:
# 플레이리스트 타이틀을 기준으로 merge해 중복된 플레이리스트만 확인
playlists_dup = pd.merge(playlists, plylst_id_cnt, on='plylst_title')
playlists_dup= playlists_dup[playlists_dup['mapping_plylst_cnt_category'] == '중복 있음']
playlists_dup['song_cnt'] = playlists_dup['songs'].apply(len)
playlists_dup['tag_cnt'] = playlists_dup['tags'].apply(len)
playlists_dup.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt,tag_cnt
16,"[감성, 질리지않는, 나만알고싶은, Pop]",1516,나만 알고싶은 노래들,"[331055, 99287, 310974, 376435, 146989, 430106...",1,2018-01-02 12:10:59.000,4,중복 있음,31,4
17,"[기분전환, 까페]",86179,나만 알고싶은 노래들,"[480142, 578916, 467225, 701072, 197046, 63557...",6,2016-10-06 02:08:40.000,4,중복 있음,46,2
18,"[숨은명곡, 좋은노래]",54673,나만 알고싶은 노래들,"[113618, 422482, 380069, 75425, 79141, 657137,...",7,2016-07-15 10:20:17.000,4,중복 있음,23,2
19,"[감성힙합, 나만알고싶은, 뿜뿜, 그루브, 노래, 감성]",135616,나만 알고싶은 노래들,"[470548, 603591, 382764, 288501, 187291, 37802...",30,2018-07-13 08:18:54.000,4,중복 있음,45,6
44,"[기분전환, 가을]",105328,트렌디한 Cafe Pop Music,"[419702, 403579, 472776, 502078, 662300, 44363...",55,2014-09-18 11:29:55.000,2,중복 있음,16,2


In [168]:
# 같은 타이틀을 가진 플레이리스트의 노래 개수 비교
# playlists_dup에서 타이틀별로 groupby 후 song_cnt 값들을 song_cnt_nunique(새 컬럼)에 리스트 형태로 넣은 후 중복을 없애기 위해 set 진행
# 1의 의미는 노래 갯수가 중복되어 있다는 뜻이기 때문에 1의 갯수가 적은 지표를 기준으로 중복 플레이리스트 제거를 진행해야함
from collections import Counter

playlists_dup_song_cnt = playlists_dup.groupby('plylst_title')['song_cnt'].apply(list).reset_index()
playlists_dup_song_cnt['song_cnt_nunique'] = playlists_dup_song_cnt['song_cnt'].apply(lambda x: len(set(x)))
Counter(playlists_dup_song_cnt['song_cnt_nunique'])

Counter({2: 1543,
         1: 510,
         3: 343,
         4: 130,
         5: 56,
         6: 47,
         7: 30,
         9: 9,
         14: 5,
         20: 2,
         8: 17,
         15: 5,
         11: 8,
         10: 5,
         16: 3,
         13: 3,
         25: 1,
         12: 2,
         19: 1,
         22: 1,
         17: 1})

In [169]:
# 같은 타이틀을 가진 플레이리스트의 태그 개수 비교

playlists_dup_tag_cnt = playlists_dup.groupby('plylst_title')['tag_cnt'].apply(list).reset_index()
playlists_dup_tag_cnt['tag_cnt_nunique'] = playlists_dup_tag_cnt['tag_cnt'].apply(lambda x: len(set(x)))
Counter(playlists_dup_tag_cnt['tag_cnt_nunique'])

Counter({1: 1225, 3: 183, 2: 1222, 4: 59, 5: 20, 6: 9, 7: 4})

In [170]:
# 같은 타이틀을 가진 플레이리스트의 좋아요 비교

playlists_dup_like_cnt = playlists_dup.groupby('plylst_title')['like_cnt'].apply(list).reset_index()
playlists_dup_like_cnt['like_cnt_nunique'] = playlists_dup_like_cnt['like_cnt'].apply(lambda x: len(set(x)))
Counter(playlists_dup_like_cnt['like_cnt_nunique'])

Counter({1: 431,
         2: 1741,
         3: 311,
         4: 102,
         5: 54,
         6: 32,
         7: 14,
         11: 4,
         9: 7,
         13: 2,
         10: 4,
         12: 5,
         8: 10,
         15: 1,
         14: 2,
         18: 1,
         24: 1})

- 플레이리스트 타이틀이 중복인 플레이리스트만 따로 모으고
- 그 중복 플리모음 중 노래갯수가 가장 많은 하나만 남겨두고
- 원래 train의 중복 없음만 따로 모아 중복모음과 concat

In [171]:
# train에 중복 갯수, 중복 여부 컬럼 추가, 플레이리스트 내 곡 갯수 추가
playlists_add_col = pd.merge(playlists, plylst_id_cnt, on='plylst_title')
playlists_add_col['song_cnt'] = playlists_add_col['songs'].apply(lambda x : len(x))

# 매핑갯수, 타이틀, 플리 내 노래 갯수, like_cnt 내림차순 정렬
playlists_filter = playlists_add_col.sort_values(['mapping_plylst_cnt', 'plylst_title', 'song_cnt', 'like_cnt'], ascending=False)
playlists_filter

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt
8532,"[집중, 어쿠스틱, 혼자, 발라드, 우울, 드라이브, 산책, 조용조용, 띵곡들, 인디]",152785,혼자만의 시간이 필요할때,"[208812, 629738, 232874, 13281, 193232, 389468...",1,2020-03-26 06:14:54.000,43,중복 있음,81
8519,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 띵곡만, 산책, 운전, 인디]",127180,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-03-24 06:39:20.000,43,중복 있음,78
8536,"[띵곡, 혼자만, 어쿠스틱, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 인디]",76448,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-04-13 05:56:55.000,43,중복 있음,78
8541,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 띵곡들, 인디]",141114,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-22 06:38:57.000,43,중복 있음,78
8548,"[어쿠스틱, 혼자, 발라드, 분위기, 드라이브, 조용한, 산책, 운전, 띵곡들, 인디]",18201,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-06 05:20:46.000,43,중복 있음,78
...,...,...,...,...,...,...,...,...,...
31846,[일렉],56144,00 Party 일렉트로댄스,"[256418, 183424, 431998, 654442, 354592, 24132...",7,2018-02-06 16:02:42.000,1,중복 없음,20
17805,[일렉],58768,00 Electronica EDM,"[673003, 251403, 617690, 441958, 436128, 62788...",5,2020-01-05 00:24:22.000,1,중복 없음,133
108585,[일렉],110536,0 일렉듣고 광질준비 O K ElecVVIP,"[349306, 93890, 651902, 46845, 362347, 330768,...",173,2012-11-29 14:34:56.000,1,중복 없음,59
52783,"[비오는날, 추억, 회상]",106660,비오는날 듣기 좋은 연주곡 모음,"[79329, 51922, 173875, 216115, 422584, 509605,...",6,2015-10-06 11:45:24.000,1,중복 없음,25


In [172]:
playlists_dup = playlists_filter[playlists_filter['mapping_plylst_cnt_category'] == '중복 있음'] # 중복 플리 모음
playlists_dup['plylst_title'].nunique() # 중복된 플레이리스트의 유니크 타이틀 갯수 : 2723

2722

In [173]:
playlists_dup['tag_cnt'] = playlists_dup['tags'].apply(len)
playlists_dup.head()

C:\Users\cg063\AppData\Local\Temp\ipykernel_14084\2448186990.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlists_dup['tag_cnt'] = playlists_dup['tags'].apply(len)


,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt,tag_cnt
8532,"[집중, 어쿠스틱, 혼자, 발라드, 우울, 드라이브, 산책, 조용조용, 띵곡들, 인디]",152785,혼자만의 시간이 필요할때,"[208812, 629738, 232874, 13281, 193232, 389468...",1,2020-03-26 06:14:54.000,43,중복 있음,81,10
8519,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 띵곡만, 산책, 운전, 인디]",127180,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-03-24 06:39:20.000,43,중복 있음,78,10
8536,"[띵곡, 혼자만, 어쿠스틱, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 인디]",76448,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-04-13 05:56:55.000,43,중복 있음,78,10
8541,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 띵곡들, 인디]",141114,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-22 06:38:57.000,43,중복 있음,78,10
8548,"[어쿠스틱, 혼자, 발라드, 분위기, 드라이브, 조용한, 산책, 운전, 띵곡들, 인디]",18201,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-06 05:20:46.000,43,중복 있음,78,10


In [174]:
# playlists_dup 중 각 플리 제목 별 노래갯수가 가장 많은 하나만 남긴다
playlists_dup_filter = pd.DataFrame(playlists_dup[playlists_dup.groupby('plylst_title')['song_cnt'].transform(max) == playlists_dup['song_cnt']]) # 3572 
playlists_dup_filter = pd.DataFrame(playlists_dup_filter[playlists_dup_filter.groupby('plylst_title')['like_cnt'].transform(max) == playlists_dup_filter['like_cnt']]) # 2928 
playlists_dup_filter = pd.DataFrame(playlists_dup_filter[playlists_dup_filter.groupby('plylst_title')['tag_cnt'].transform(max) == playlists_dup_filter['tag_cnt']]) # 2909 
playlists_dup_filter = pd.DataFrame(playlists_dup_filter[playlists_dup_filter.groupby('plylst_title')['updt_date'].transform(max) == playlists_dup_filter['updt_date']]) # 2748 
playlists_dup_filter

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt,tag_cnt
8532,"[집중, 어쿠스틱, 혼자, 발라드, 우울, 드라이브, 산책, 조용조용, 띵곡들, 인디]",152785,혼자만의 시간이 필요할때,"[208812, 629738, 232874, 13281, 193232, 389468...",1,2020-03-26 06:14:54.000,43,중복 있음,81,10
3119,[비오는날듣기좋은노래],75012,비오는날,"[65672, 522825, 145252, 146780, 500731, 473608...",22,2017-08-19 15:16:36.000,34,중복 있음,200,1
5536,"[조용히, 어쿠스틱, 여유, 발라드, 분위기, 힐링, 자기전이나, 띵곡, 인디, 편안한]",52312,자기전 하루한곡 발라드,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-04-02 06:54:25.000,31,중복 있음,82,10
8429,[금요일],48129,금요일,"[614756, 147427, 340332, 675466, 414036, 61619...",0,2017-03-10 17:31:34.000,30,중복 있음,20,1
7288,[발라드],5960,추억,"[208821, 481749, 565402, 362197, 407236, 46604...",10,2010-10-21 09:45:12.000,27,중복 있음,85,1
...,...,...,...,...,...,...,...,...,...,...
155,"[기분좋은, 100번]",148495,100번 들어도 기분좋은 가요,"[208186, 155952, 669617, 438439, 397412, 60398...",4,2016-10-31 13:59:41.000,2,중복 있음,16,2
76640,[Pop],62278,100% 로맨틱 조지 마이클 223,"[366485, 435804, 442680, 538949, 263360, 29495...",0,2019-03-27 15:27:44.000,2,중복 있음,34,1
34444,[팝],50564,1 HITS OF DECADE SPECIAL 2,"[60025, 326802, 82991, 533287, 404318, 82686, ...",5,2009-07-11 21:37:42.000,2,중복 있음,37,1
75854,[팝],109558,1 HITS OF DECADE SPECIAL,"[289618, 78629, 349460, 218356, 189583, 613209...",5,2009-07-05 18:23:47.000,2,중복 있음,39,1


In [175]:
playlists_dup_filter[playlists_dup_filter['plylst_title'] == 'qqqa'] # 필터링 되지 않은 중복 타이틀은 플레이리스트 자체가 같음

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt,tag_cnt
50442,[여름],7524,qqqa,"[522825, 455668, 549178, 407828, 343974]",1,2015-11-04 17:51:14.000,3,중복 있음,5,1
50443,[여름],43031,qqqa,"[522825, 455668, 549178, 407828, 343974]",1,2015-11-04 17:51:14.000,3,중복 있음,5,1


In [176]:
# 타이틀이 같은 플레이리스트 중 하나만 남긴다
playlists_dup_filter = pd.DataFrame(playlists_dup_filter.groupby('plylst_title', group_keys=False).apply(lambda x: x.loc[x['song_cnt'].idxmax()]))
playlists_dup_filter = playlists_dup_filter.reset_index(drop=True) # 2723 

In [177]:
# playlists_dup_filter 중 중복 여부 확인

# 1. 플레이리스트 아이디(id)와 플레이리스트 명(plylst_title) 추출
plylst_title_2 = playlists_dup_filter[['id', 'plylst_title']]

# 2. 플레이리스트 이름 별 플레이리스트 ID count 테이블 생성 : plylst_id_cnt
plylst_id_cnt_2 = plylst_title_2.groupby('plylst_title').id.nunique().reset_index(name = 'mapping_plylst_cnt')

# 3. 플레이리스트 명 중복 구분 : 서로 다른 플레이리스트 아이디로 겹치는 플레이리스트 이름이 없으면 '중복 없음', 그 외 '중복 있음'
plylst_id_cnt_2 = plylst_id_cnt_2.assign(
    mapping_plylst_cnt_category = pd.cut(plylst_id_cnt_2['mapping_plylst_cnt'], [0, 1, np.inf], labels = ['중복 없음', '중복 있음'])
)
plylst_id_cnt_2
# [0, 1, np.inf] : 0이상 1미만 값 = 중복없음, 1이상의 값 = 중복있음

# 4. 중복 구분 별 플레이리스트 수 count 테이블 생성 : plylst_id_cnt_division
plylst_id_cnt_division_2 = pd.DataFrame(plylst_id_cnt_2.groupby('mapping_plylst_cnt_category').count())
plylst_id_cnt_division_2

,plylst_title,mapping_plylst_cnt
mapping_plylst_cnt_category,,
중복 없음,2722,2722
중복 있음,0,0


In [180]:
# 중복 없음 파일만 만들기
playlists_no_dup = pd.merge(playlists, plylst_id_cnt, on='plylst_title')
playlists_no_dup = playlists_no_dup[playlists_no_dup['mapping_plylst_cnt_category'] == '중복 없음']
playlists_no_dup # 106525행

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,중복 없음
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,1,중복 없음
2,"[까페, 잔잔한]",76951,편하게 잔잔하게 들을 수 있는 곡,"[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,1,중복 없음
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,1,중복 없음
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,1,중복 없음
...,...,...,...,...,...,...,...,...
114702,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E SM 2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,1,중복 없음
114703,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,1,중복 없음
114704,"[담시, 가족, 눈물, 그리움, 주인공, 나의이야기, 사랑, 친구]",11343,1 눈물이 앞을 가리는 나의 이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,1,중복 없음
114705,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,1,중복 없음


In [183]:
playlists_dup_proc = pd.concat([playlists_dup_filter, playlists_no_dup], axis=0)
playlists_dup_proc # 2722+106525 =109247

# 114707 - 109247 = 타이틀 중복 제거로 5460행 삭제

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt,tag_cnt
0,[Pop],123815,1 1 환상의 케미스트리 211,"[243181, 520307, 291514, 496987, 285795, 68548...",0,2019-03-27 15:27:43.000,2,중복 있음,36.0,1.0
1,[팝],109558,1 HITS OF DECADE SPECIAL,"[289618, 78629, 349460, 218356, 189583, 613209...",5,2009-07-05 18:23:47.000,2,중복 있음,39.0,1.0
2,[팝],50564,1 HITS OF DECADE SPECIAL 2,"[60025, 326802, 82991, 533287, 404318, 82686, ...",5,2009-07-11 21:37:42.000,2,중복 있음,37.0,1.0
3,[Pop],62278,100% 로맨틱 조지 마이클 223,"[366485, 435804, 442680, 538949, 263360, 29495...",0,2019-03-27 15:27:44.000,2,중복 있음,34.0,1.0
4,"[기분좋은, 100번]",148495,100번 들어도 기분좋은 가요,"[208186, 155952, 669617, 438439, 397412, 60398...",4,2016-10-31 13:59:41.000,2,중복 있음,16.0,2.0
...,...,...,...,...,...,...,...,...,...,...
114702,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E SM 2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,1,중복 없음,NaN,NaN
114703,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,1,중복 없음,NaN,NaN
114704,"[담시, 가족, 눈물, 그리움, 주인공, 나의이야기, 사랑, 친구]",11343,1 눈물이 앞을 가리는 나의 이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,1,중복 없음,NaN,NaN
114705,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,1,중복 없음,NaN,NaN


> 3. 플레이리스트 별 노래 개수, 좋아요 개수 이상치 행 날리기(태그는 보고 진행)

- 노래 개수 5~106개 가진 플레이리스트만 남기기

In [188]:
playlists = playlists_dup_proc.iloc[:, :5]
playlists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109247 entries, 0 to 114706
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          109247 non-null  object
 1   id            109247 non-null  int64 
 2   plylst_title  109247 non-null  object
 3   songs         109247 non-null  object
 4   like_cnt      109247 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 5.0+ MB


In [191]:
playlists['songs_cnt'] = playlists['songs'].apply(lambda x : len(x))
playlists_song_cut = playlists[(playlists['songs_cnt'] >= 5) & (playlists['songs_cnt'] < 106) ]  # 5~106개만 남기기
playlists_song_cut.info() # 10139행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99108 entries, 0 to 114705
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          99108 non-null  object
 1   id            99108 non-null  int64 
 2   plylst_title  99108 non-null  object
 3   songs         99108 non-null  object
 4   like_cnt      99108 non-null  int64 
 5   songs_cnt     99108 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 5.3+ MB


- 좋아요 2개 이하(describe 시 25%가 2개임) 삭제

In [18]:
playlists_song_cut['like_cnt'].describe()

count    103702.000000
mean         73.147750
std         418.398668
min           0.000000
25%           2.000000
50%           7.000000
75%          20.000000
max       53211.000000
Name: like_cnt, dtype: float64

In [19]:
playlists_song_tag_cut = playlists_song_cut[playlists_song_cut['like_cnt'] > 2]
playlists_song_tag_cut.info()# 27852행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75850 entries, 0 to 115069
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          75850 non-null  object
 1   id            75850 non-null  int64 
 2   plylst_title  75850 non-null  object
 3   songs         75850 non-null  object
 4   like_cnt      75850 non-null  int64 
 5   updt_date     75850 non-null  object
dtypes: int64(2), object(4)
memory usage: 4.1+ MB


> 3. song, 태그 빈도수에 따라 리스트에서 빼기 -> 리스트가 비워져있으면 행 날리기

- 1회 이하 사용된 tag 리스트에서 빼기

In [21]:
playlists_song_tag_cut

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
2,"[까페, 잔잔한]",76951,편하게 잔잔하게 들을 수 있는 곡,"[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000
5,"[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...",69252,2017 Pop Trend,"[418694, 222305, 96545, 135950, 304687, 457451...",435,2017-09-15 15:59:26.000
...,...,...,...,...,...,...
115065,"[설렘, 사랑]",135687,남자고백노래,"[287003, 408232, 625933, 152186, 543298, 22271...",14,2016-06-23 13:24:27.000
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E SM 2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의이야기, 사랑, 친구]",11343,1 눈물이 앞을 가리는 나의 이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000


In [29]:
# tag usage count 계산
tag_counts = playlists_song_tag_cut.explode('tags')['tags'].value_counts(ascending=True)

# 1번 이하로 사용된 태그
unique_tags = list(tag_counts[tag_counts == 1].index)
unique_tags

아이돌학교             2
새벽에듣기좋은           2
한번쯤은들어봤을          2
markronson        2
박주원               2
              ...  
힐링             6549
드라이브           6875
휴식             7165
감성             7274
기분전환          10641
Name: tags, Length: 9346, dtype: int64

In [ ]:
# 5번 이하로 사용된 태그
unique_tags = list(tag_counts[tag_counts <= 5].index)
unique_tags

In [28]:
# tag_counts를 집합(set)으로 변환
unique_tags = set(tag_counts)
len(unique_tags)

333

In [24]:
filtered_tags = tag_counts[tag_counts > 1].index

In [25]:
playlists_tags_filter = playlists_song_tag_cut.explode('tags')
playlists_tags_filter = playlists_tags_filter[playlists_tags_filter['tags'].isin(filtered_tags)]

playlists_tags_filter = playlists_tags_filter.groupby('id').agg({
    'tags': 'first',
    'plylst_title': 'first',
    'songs': list,
    'like_cnt': 'first',
    'updt_date': 'first'
}).reset_index()

In [26]:
# tags가 비어있지 않은 행만 남김
playlists_tags_filter = playlists_tags_filter[playlists_tags_filter['tags'].apply(lambda x: len(x) > 0)]
playlists_tags_filter.reset_index(drop=True)
playlists_tags_filter.info() # 255행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52471 entries, 0 to 52473
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            52471 non-null  int64 
 1   tags          52471 non-null  object
 2   plylst_title  52471 non-null  object
 3   songs         52471 non-null  object
 4   like_cnt      52471 non-null  int64 
 5   updt_date     52471 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.8+ MB


- 2회 이하 사용된 song 리스트에서 빼기

In [27]:
# Song usage count 계산
song_counts = playlists_tags_filter.explode('songs')['songs'].value_counts()

In [28]:
# 2회 이하로 사용된 song을 필터링
filtered_songs = song_counts[song_counts > 2].index

In [29]:
playlists_songs_filter = playlists_tags_filter.explode('songs')
playlists_songs_filter = playlists_songs_filter[playlists_songs_filter['songs'].isin(filtered_songs)]

In [30]:
playlists_songs_filter = playlists_songs_filter.groupby('id').agg({
    'tags': 'first',
    'plylst_title': 'first',
    'songs': list,
    'like_cnt': 'first',
    'updt_date': 'first'
}).reset_index()

In [31]:
# songs가 비어있지 않은 행만 남김
playlists_songs_filter = playlists_songs_filter[playlists_songs_filter['songs'].apply(lambda x: len(x) > 0)]
playlists_songs_filter.reset_index(drop=True)
playlists_songs_filter.info() # 20094행 삭제...?

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32377 entries, 0 to 32376
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            32377 non-null  int64 
 1   tags          32377 non-null  object
 2   plylst_title  32377 non-null  object
 3   songs         32377 non-null  object
 4   like_cnt      32377 non-null  int64 
 5   updt_date     32377 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.7+ MB


In [34]:
playlists_songs_filter.explode('tags')['tags'].value_counts() 

감성         1892
힙합         1683
휴식         1526
힐링         1423
비오는날       1013
           ... 
포근한음악         1
긍정적인마인드       1
형제            1
오스트           1
외출            1
Name: tags, Length: 3055, dtype: int64

In [41]:
# 알맞게 제거되었는지 확인
#playlists_songs_filter.explode('tags')['tags'].value_counts() # -> 다시 생김....?
playlists_songs_filter.explode('songs')['songs'].value_counts()

[96843, 250823, 507298, 487867, 650859, 630941, 654833, 8014, 146525, 560373, 367736, 213364, 542446, 6888, 611069]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       145
[35848, 638330, 132604, 666824, 231868, 546740, 483630, 616798, 453477, 431049]                                                                                                                                                                                                           

In [127]:
# tag_counts = playlists_songs_filter.explode('tags')['tags'].value_counts(ascending=True)
# # 1회 이하로 사용된 tags 필터링
# filtered_tags = tag_counts[tag_counts > 1].index
# filtered_tags

Index(['주말엔', '이바다', '애매한사이', 'OnoYoko', 'JohnLennon', '스위트박스', '원조아이돌',
       '90년대댄스', '어쿠스틱한', '누메탈',
       ...
       '밤', '사랑', '발라드', '새벽', '힐링', '잔잔한', '휴식', '감성', '드라이브', '기분전환'],
      dtype='object', length=7587)

In [129]:
# playlists_tags_filter = playlists_songs_filter.explode('tags')
# playlists_tags_filter = playlists_tags_filter[playlists_tags_filter['tags'].isin(filtered_tags)]

# playlists_tags_filter = playlists_tags_filter.groupby('id').agg({
#     'tags': 'first',
#     'plylst_title': 'first',
#     'songs': list,
#     'like_cnt': 'first',
#     'updt_date': 'first'
# }).reset_index()

In [130]:
# # tags가 비어있지 않은 행만 남김
# playlists_tags_filter = playlists_tags_filter[playlists_tags_filter['tags'].apply(lambda x: len(x) > 0)]
# playlists_tags_filter.reset_index(drop=True)
# playlists_tags_filter.info() # 248행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51881 entries, 0 to 51883
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            51881 non-null  int64 
 1   tags          51881 non-null  object
 2   plylst_title  51881 non-null  object
 3   songs         51881 non-null  object
 4   like_cnt      51881 non-null  int64 
 5   updt_date     51881 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.8+ MB


In [134]:
# # 알맞게 제거되었는지 확인
# playlists_tags_filter.explode('tags')['tags'].value_counts() # -> 다시 생김
# playlists_tags_filter.explode('songs')['songs'].value_counts()

=> 19029 개 제거로 57280개의 플레이리스트만 남음

> 데이터 저장

In [ ]:
df.to_json('../0_data/data.json', orient='records')

- 최종 전처리 파일은 ../0_data/data.json 으로 저장

---